In [ ]:
from pathlib import Path
import sys

import pandas as pd
import numpy as np

sys.path.append('../')
import payulator as pl


%load_ext autoreload
%autoreload 2

In [2]:
loan = pl.Loan(interest_rate=0.12)
print(loan)
loan.summarize()['payment_schedule']

In [16]:
params = {
    'code': 'test-20180324',
    'principal': 10000,
    'interest_rate': 0.12,
    'kind': 'amortized',
    'num_payments': 6,
    'payment_freq': 'monthly',
    'start_date': '2018-03-24',
    'fee': 250,
    'lender': 'Bingo',
    'borrowers': ['Bongo', 'Boom'],
}

params = pl.check_loan_params(params)
params = pl.prune_loan_params(params)
loan = pl.Loan(**params)
print(loan)

code: test-20180324
kind: amortized
principal: 10000
interest_rate: 0.12
payment_freq: monthly
compounding_freq: monthly
num_payments: 6
start_date: 2018-03-24
fee: 250


In [14]:
loan.summarize()

{'interest_and_fee_total': 602.9,
 'interest_and_fee_total/principal': 0.06,
 'interest_total': 352.9,
 'payment_schedule':    payment_sequence payment_date  beginning_balance  principal_payment  \
 0                 1   2018-03-24           10000.00            1625.48   
 1                 2   2018-04-24            8374.52            1641.74   
 2                 3   2018-05-24            6732.78            1658.16   
 3                 4   2018-06-24            5074.62            1674.74   
 4                 5   2018-07-24            3399.88            1691.48   
 5                 6   2018-08-24            1708.40            1708.40   
 
    interest_payment  ending_balance  notes  
 0            100.00         8374.52    NaN  
 1             83.75         6732.78    NaN  
 2             67.33         5074.62    NaN  
 3             50.75         3399.88    NaN  
 4             34.00         1708.40    NaN  
 5             17.08            0.00    NaN  ,
 'payment_total': 10602.9,


In [17]:
params = {
    'code': 'test2-20180324',
    'kind': 'interest_only',
    'principal': 10000,
    'interest_rate': 0.12,
    'num_payments': 6,
    'payment_freq': 'monthly',
    'start_date': '2018-03-24',
    'fee': 250,
    'lender': 'Bingo',
    'borrowers': ['Bongo', 'Boom'],
}

params = pl.check_loan_params(params)
params = pl.prune_loan_params(params)
loan = pl.Loan(**params)
print(loan)

code: test2-20180324
kind: interest_only
principal: 10000
interest_rate: 0.12
payment_freq: monthly
compounding_freq: monthly
num_payments: 6
start_date: 2018-03-24
fee: 250


In [18]:
loan.summarize()

{'interest_and_fee_total': 850.0,
 'interest_and_fee_total/principal': 0.08,
 'interest_total': 600.0,
 'payment_schedule':    payment_sequence payment_date  beginning_balance  principal_payment  \
 0                 1   2018-03-24              10000                  0   
 1                 2   2018-04-24              10000                  0   
 2                 3   2018-05-24              10000                  0   
 3                 4   2018-06-24              10000                  0   
 4                 5   2018-07-24              10000                  0   
 5                 6   2018-08-24              10000              10000   
 
    interest_payment  ending_balance  notes  
 0             100.0           10000    NaN  
 1             100.0           10000    NaN  
 2             100.0           10000    NaN  
 3             100.0           10000    NaN  
 4             100.0           10000    NaN  
 5             100.0               0    NaN  ,
 'payment_total': 10850.0}